In [4]:
import altair as alt
import pandas as pd
alt.renderers.set_embed_options(theme='dark')

hits = pd.read_csv('../data/processed/audio_data_processed.csv')
hits.drop(["Unnamed: 0",'type','uri','track_href','analysis_url','id'], axis=1, inplace=True)
hits['rank_bin'] = pd.cut(hits['Rank'],bins=10,labels=['1-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100'])

hits_c = hits
hits_c_bin = hits_c.groupby('rank_bin').mean().reset_index()
hits_c_bin = hits_c_bin[['rank_bin','popularity','energy','speechiness','instrumentalness','valence']].set_index(['rank_bin','popularity']).stack().reset_index(name='value').rename(columns={'level_2':'features'})
charts=[]
titleParams=[alt.TitleParams(text = x,subtitle=y,anchor='start',fontSize = 24) for (x,y) in [
            ("Vibe Features","Energy, speechiness, instrumentalness,valence in different strata of the charts"),
            ("Rhythm Features", "Time signature, tempo, duration occurences"),
            ("Musical Features", "Musical key, mode occurences")]]

for i in ['energy','speechiness','instrumentalness','valence']:
    charts.append(alt.Chart(hits_c_bin[hits_c_bin['features'] == i]).mark_square().encode(
    x='rank_bin:N',
    y=alt.Y('value',scale=alt.Scale(zero=False)),
    color='features:N',
    opacity='popularity:Q',
    size= 'popularity:Q'
    ))
(charts[0]+charts[1]+charts[2]+charts[3]).properties(
        width=900,height=250, title = titleParams[0])

alt.LayerChart(...)

In [2]:
from lyrics_rub import *
lr = LyricsRub()
lyrics_str, lyrics_line_lst = lr.process_lyrics(lr.get_lyrics('space%20oddity%20david%20bowie'))
print(lyrics_str)
print(lyrics_line_lst)

ModuleNotFoundError: No module named 'lyrics_rub'

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import os
import json
import re
import yaml
headers={
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}
#for crawling
with open('config.yml', "r") as f:
    client = yaml.safe_load(f)
apikey = client['apikey']
ss = requests.Session()
req = ss.get(f'http://api.musixmatch.com/ws/1.1/track.search?apikey={self.apikey}&q={q}&page_size=3&page=1&s_track_rating=desc')
reqt= json.loads(req.text)